In [14]:
from azureml.core import Workspace, Dataset, Datastore, Experiment, Run



In [13]:
ws = Workspace.from_config()
az_store = Datastore.get(ws, "azure_sdk_nse_blob")
az_dataset = Dataset.get_by_name(ws, "final_data_stocks")



In [ ]:
new_run = Run.get_context()

In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error

df = az_dataset.to_pandas_dataframe()
df.set_index("Date", inplace=True)
# Extract individual log return series
stocks = ["RELIANCE_log_returns", "TCS_log_returns", "HDFCBANK_log_returns"]

# Train-Test Split Ratio
train_ratio = 0.8

# Dictionary to store model results
results = {}

# Function to fit ARIMA model
def fit_arima_model(train, test, stock_name):
    # Order of ARIMA (p,d,q) - can be tuned using AIC/BIC or auto_arima
    order = (1, 1, 1)  # Example order, can be optimized

    # Train ARIMA model
    model = ARIMA(train, order=order)
    model_fit = model.fit()

    # Forecast
    forecast = model_fit.forecast(steps=len(test))

    # Compute evaluation metrics
    mae = mean_absolute_error(test, forecast)
    rmse = np.sqrt(mean_squared_error(test, forecast))

    # Store results
    results[stock_name] = {"model": model_fit, "forecast": forecast, "mae": mae, "rmse": rmse}

    """# Plot actual vs forecast
    plt.figure(figsize=(10, 4))
    plt.plot(test.index, test, label="Actual")
    plt.plot(test.index, forecast, label="Forecast", linestyle="dashed")
    plt.title(f"ARIMA Forecast for {stock_name}")
    plt.legend()
    plt.show()"""

    print(f"Stock: {stock_name}")
    print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}\n")

# Apply ARIMA to each stock's log returns
for stock in stocks:
    series = df[stock].dropna()  # Remove NaN values if any

    # Train-Test Split
    train_size = int(len(series) * train_ratio)
    train, test = series[:train_size], series[train_size:]

    # Fit ARIMA Model
    fit_arima_model(train, test, stock)

# Display overall results
print("ARIMA Model Results:")
for stock, res in results.items():
    print(f"{stock}: MAE = {res['mae']:.4f}, RMSE = {res['rmse']:.4f}")


In [ ]:

new_run.log("ARIMA Model Results:")
for stock, res in results.items():
    new_run.log(f"{stock}: MAE = {res['mae']:.4f}, RMSE = {res['rmse']:.4f}")

In [7]:
#experiment2 = Experiment(workspace=ws, name="Stocks-exp02")

In [8]:
#running an experiment
#new_run = experiment.start_logging(snapshot_directory=None)

In [9]:
#df = az_dataset.to_pandas_dataframe()
#total_observations = len(df)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [10]:
#new_run.log("total observations", total_observations)

In [11]:
new_run.complete()